In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
import numpy as np
from tensorflow.python.platform import gfile

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
f = gfile.FastGFile("../Personal/cat-vs-dog/Keras-Cat-vs-Dog-RTN.pb",'rb')
graph_def = tf.GraphDef()
graph_def.ParseFromString(f.read())
tf.import_graph_def(graph_def, name='')

Device mapping: no known devices.
Instructions for updating:
Use tf.gfile.GFile.


In [4]:
node = 0
for n in tf.get_default_graph().as_graph_def().node:
    print(n.name)
    node = node+1

input_layer
conv2d/kernel
conv2d/bias
conv2d/Conv2D
conv2d/BiasAdd
re_lu/Relu
batch_normalization/gamma
batch_normalization/beta
batch_normalization/moving_mean
batch_normalization/moving_variance
keras_learning_phase/input
keras_learning_phase
batch_normalization/cond/ReadVariableOp/Switch
batch_normalization/cond/ReadVariableOp_1/Switch
batch_normalization/cond/FusedBatchNormV3/ReadVariableOp/Switch
batch_normalization/cond/FusedBatchNormV3/ReadVariableOp_1/Switch
batch_normalization/cond/FusedBatchNormV3/Switch
batch_normalization/cond/FusedBatchNormV3
batch_normalization/cond/ReadVariableOp_2/Switch
batch_normalization/cond/ReadVariableOp_3/Switch
batch_normalization/cond/FusedBatchNormV3_1/ReadVariableOp/Switch
batch_normalization/cond/FusedBatchNormV3_1/ReadVariableOp_1/Switch
batch_normalization/cond/FusedBatchNormV3_1/Switch
batch_normalization/cond/FusedBatchNormV3_1
batch_normalization/cond/Merge
dropout/cond/Switch
dropout/cond/switch_t
dropout/cond/dropout/Const
dropout/con

In [6]:
node

277

In [11]:
from tensorflow.python.tools.optimize_for_inference_lib import node_name_from_input, node_from_map


In [13]:
input_node_map = {}
for node in graph_def.node:
    if node.name not in input_node_map:
        input_node_map[node.name] = node
    else:
        raise ValueError("Duplicate node names detected for ", node.name)

In [16]:
for node in graph_def.node:
    for idx, i in enumerate(node.input):
        print(i)
        input_clean = node_name_from_input(i)
        if input_clean.endswith('/cond/Merge') and input_clean.split('/')[-3].startswith('dropout'):
            identity = node_from_map(input_node_map, i).input[0]
#             assert identity.split('/')[-1] == 'Identity'
            parent = node_from_map(input_node_map, node_from_map(input_node_map, identity).input[0])
            pred_id = parent.input[1]
#             assert pred_id.split('/')[-1] == 'pred_id'            
            good = parent.input[0]
            node.input[idx] = good

input_layer
conv2d/kernel
conv2d/Conv2D
conv2d/bias
conv2d/BiasAdd
keras_learning_phase/input
batch_normalization/gamma
keras_learning_phase
batch_normalization/beta
keras_learning_phase
batch_normalization/moving_mean
keras_learning_phase
batch_normalization/moving_variance
keras_learning_phase
batch_normalization/cond/FusedBatchNormV3/Switch:1
batch_normalization/cond/ReadVariableOp/Switch:1
batch_normalization/cond/ReadVariableOp_1/Switch:1
batch_normalization/cond/FusedBatchNormV3/ReadVariableOp/Switch:1
batch_normalization/cond/FusedBatchNormV3/ReadVariableOp_1/Switch:1
re_lu/Relu
keras_learning_phase
batch_normalization/gamma
keras_learning_phase
batch_normalization/beta
keras_learning_phase
batch_normalization/moving_mean
keras_learning_phase
batch_normalization/moving_variance
keras_learning_phase
batch_normalization/cond/FusedBatchNormV3_1/Switch
batch_normalization/cond/ReadVariableOp_2/Switch
batch_normalization/cond/ReadVariableOp_3/Switch
batch_normalization/cond/FusedBatc

In [17]:
node = 0
for n in graph_def.node:
    print(n.name)
    node = node+1

input_layer
conv2d/kernel
conv2d/bias
conv2d/Conv2D
conv2d/BiasAdd
re_lu/Relu
batch_normalization/gamma
batch_normalization/beta
batch_normalization/moving_mean
batch_normalization/moving_variance
keras_learning_phase/input
keras_learning_phase
batch_normalization/cond/ReadVariableOp/Switch
batch_normalization/cond/ReadVariableOp_1/Switch
batch_normalization/cond/FusedBatchNormV3/ReadVariableOp/Switch
batch_normalization/cond/FusedBatchNormV3/ReadVariableOp_1/Switch
batch_normalization/cond/FusedBatchNormV3
batch_normalization/cond/FusedBatchNormV3/Switch
batch_normalization/cond/ReadVariableOp_2/Switch
batch_normalization/cond/ReadVariableOp_3/Switch
batch_normalization/cond/FusedBatchNormV3_1/ReadVariableOp/Switch
batch_normalization/cond/FusedBatchNormV3_1/ReadVariableOp_1/Switch
batch_normalization/cond/FusedBatchNormV3_1
batch_normalization/cond/FusedBatchNormV3_1/Switch
batch_normalization/cond/Merge
dropout/cond/Switch
dropout/cond/switch_t
dropout/cond/dropout/Const
dropout/con

In [18]:
node

277

In [ ]:
input_layer = sess.graph.get_tensor_by_name('input_layer:0')
keep_prob = sess.graph.get_tensor_by_name('keep_prob:0')
is_training = sess.graph.get_tensor_by_name('is_training:0')
output = sess.graph.get_tensor_by_name('output_layer:0')

In [ ]:
sess.run(output, feed_dict={input_layer : np.random.randint(255,size=(4,128,128,3))/255 - 0.5, keep_prob : 1.0, \
                           is_training : True})

In [ ]:
sess.run(output, feed_dict={input_layer : np.random.randint(255,size=(4,128,128,3))/255 - 0.5, keep_prob : 1.0, \
                           is_training : True})

In [ ]:
tf.__version__

In [ ]:
import tensorflow as tf
graph_def = tf.GraphDef()
with tf.gfile.GFile("../Personal/cat-vs-dog/AjNet_output_graph.pb", 'rb') as f:
    graph_def.ParseFromString(f.read())
from tensorflow.python.compiler.tensorrt import trt_convert as trt
converter = trt.TrtGraphConverter(input_graph_def=graph_def, nodes_blacklist=['output_layer'])
graph_def = converter.convert()